In [1]:
import pandas as np
df = np.read_csv('mbti_1.csv')

# populating types array
types = []
for _type in df['type']:
     types.append(_type)


In [2]:
import pickle

tokenized_posts = []
with open ('tokenized_formatted_data.txt', 'rb') as fp:
    tokenized_posts = pickle.load(fp)

In [3]:
import random
random.seed(673)
random.shuffle(types)
random.seed(673)
random.shuffle(tokenized_posts)

In [4]:
import pickle

with open ('synthetic_data.pkl', 'rb') as fp:
    synthetic_data = pickle.load(fp)
    
for labels, users in synthetic_data:
    for i in range(0, len(labels)):
        types.append(labels[i])
        tokenized_posts.append(users[i])

In [5]:
len(types)

29312

In [6]:
all_words_list = []

for user in tokenized_posts:
    for word in user:
        all_words_list.append(word)

In [7]:
from collections import Counter

In [8]:
freq_list = Counter(all_words_list)
dictionary = freq_list.most_common(10000)

In [9]:
dictionary = list(zip(*dictionary))[0]

In [10]:
nums = range(0, 10000)
word_int = dict(zip(dictionary, nums))

In [11]:
x_vals = []
for user in tokenized_posts:
    x_vals.append([word_int[x] for x in user if x in word_int.keys()])

In [13]:
x_train = x_vals[1000:]
y_train = types[1000:]

x_test = x_vals[:1000]
y_test = types[:1000]

In [14]:
type_dictionary = {
    '0000':'INTJ',
    '0001':'INTP',
    '0010':'INFJ',
    '0011':'INFP',
    '0100':'ISTJ',
    '0101':'ISTP',
    '0110':'ISFJ',
    '0111':'ISFP',
    '1000':'ENTJ',
    '1001':'ENTP',
    '1010':'ENFJ',
    '1011':'ENFP',
    '1100':'ESTJ',
    '1101':'ESTP',
    '1110':'ESFJ',
    '1111':'ESFP',
}

type_labels=['INTJ','INTP','INFJ','INFP','ISTJ','ISTP','ISFJ','ISFP', \
        'ENTJ','ENTP','ENFJ','ENFP','ESTJ','ESTP','ESFJ','ESFP',]

def to_one_hot(arr):
    one_hot_types = []

    for label in arr:
        bin_type = []

        if (label[0] == 'I'):
            bin_type.append(0)
        else:
            bin_type.append(1)

        if (label[1] == 'N'):
            bin_type.append(0)
        else:
            bin_type.append(1)

        if (label[2] == 'T'):
            bin_type.append(0)
        else:
            bin_type.append(1)

        if (label[3] == 'J'):
            bin_type.append(0)
        else:
            bin_type.append(1)

        one_hot_types.append(bin_type)
        
    return one_hot_types
    

In [15]:
import numpy as np
from keras.utils.np_utils import to_categorical

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_test)

y_train = np.asarray(to_one_hot(y_train)).astype('float32')
y_test = np.asarray(to_one_hot(y_test)).astype('float32')

Using TensorFlow backend.


In [16]:
class_weights = {
    0:1.,
    1:3.03,
    2:1.,
    3:5.35,
    4:1.,
    5:1.12,
    6:1.,
    7:0.69
}

In [17]:
# FEED FORWARD MODEL
from keras import models
from keras import layers
from keras import regularizers
import tensorflow as tf
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                           activation='relu', input_shape = (10000,)))
    model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                           activation='relu'))
    model.add(layers.Dense(4, activation='sigmoid'))
    model.compile(optimizer = 'rmsprop',
                  loss = 'binary_crossentropy',
                  metrics = ['acc'])
    return model

In [18]:
model = build_model()
with tf.device('/gpu:0'):
    history = model.fit(x_train, y_train, epochs = 10, batch_size = 128)
    y_pred = model.predict(x_test)
    results = model.evaluate(x_test, y_test)
results

Epoch 1/10
28312/28312 [==============================] - 6s 212us/step - loss: 0.4266 - acc: 0.8378
Epoch 2/10
28312/28312 [==============================] - 3s 103us/step - loss: 0.3059 - acc: 0.9084
Epoch 3/10
28312/28312 [==============================] - 3s 97us/step - loss: 0.2737 - acc: 0.9224
Epoch 4/10
28312/28312 [==============================] - 3s 97us/step - loss: 0.2568 - acc: 0.9301
Epoch 5/10
28312/28312 [==============================] - 3s 96us/step - loss: 0.2453 - acc: 0.9349
Epoch 6/10
28312/28312 [==============================] - 3s 98us/step - loss: 0.2366 - acc: 0.9382
Epoch 7/10
28312/28312 [==============================] - 3s 97us/step - loss: 0.2289 - acc: 0.9429
Epoch 8/10
28312/28312 [==============================] - 3s 98us/step - loss: 0.2225 - acc: 0.9459
Epoch 9/10
28312/28312 [==============================] - 3s 97us/step - loss: 0.2168 - acc: 0.9490
Epoch 10/10
1000/1000 [==============================] - 0s 128us/step


[0.6598332810401917, 0.762]

In [19]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import datetime

now = datetime.datetime.now()
pictures = []
folder = "FF"

def byte_to_mbti(byte):
    binary = ''
    for letter in byte:
         binary +=(str(int(letter)))        
    return type_dictionary[binary]

decoded_y_true = [byte_to_mbti(label) for label in y_test.round()]
decoded_y_pred = [byte_to_mbti(label) for label in y_pred.round()]

matrix = confusion_matrix(decoded_y_true, decoded_y_pred, labels=type_labels)
matrix_I_E = confusion_matrix(y_test[:,0], y_pred[:,0].round(), labels=[0, 1])
matrix_N_S = confusion_matrix(y_test[:,1], y_pred[:,1].round(), labels=[0, 1])
matrix_T_F = confusion_matrix(y_test[:,2], y_pred[:,2].round(), labels=[0, 1])
matrix_J_P = confusion_matrix(y_test[:,3], y_pred[:,3].round(), labels=[0, 1])

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plt.figure(figsize=(15,10))
plot_confusion_matrix(matrix, classes=type_labels, normalize=True,
                      title='Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM0.png"
plt.savefig(name)
plt.show()
pictures.append(name)

plt.clf()
plt.figure(figsize=(5,5))
plot_confusion_matrix(matrix_I_E, classes=["I", "E"], normalize=True,
                      title='I-E Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM1.png"
plt.savefig(name)
plt.show()
pictures.append(name)

plt.clf()
plt.figure(figsize=(5,5))
plot_confusion_matrix(matrix_N_S, classes=["N", "S"], normalize=True,
                      title='N-S Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM2.png"
plt.savefig(name)
plt.show()
pictures.append(name)

plt.clf()
plt.figure(figsize=(5,5))
plot_confusion_matrix(matrix_T_F, classes=["T", "F"], normalize=True,
                      title='T-F Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM3.png"
plt.savefig(name)
plt.show()
pictures.append(name)

plt.clf()
plt.figure(figsize=(5,5))
plot_confusion_matrix(matrix_J_P, classes=["J", "P"], normalize=True,
                      title='J-P Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM4.png"
plt.savefig(name)
plt.show()
pictures.append(name)

Normalized confusion matrix


<Figure size 1500x1000 with 2 Axes>

Normalized confusion matrix


<Figure size 640x480 with 0 Axes>

<Figure size 500x500 with 2 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 500x500 with 2 Axes>

Normalized confusion matrix


<Figure size 640x480 with 0 Axes>

<Figure size 500x500 with 2 Axes>

In [20]:
def myprint(s):
    with open('results.log','a+') as f:
        print(s, file=f)
    f.close()
        
log_file = open('results.log', 'a+')

log_file.write("\n=================================================================")
log_file.write( "\n" + str(now.month) + "/" + str(now.day) + "/" + str(now.year) + " " + \
str(now.hour) + ":" + str(now.minute) + ":" + str(now.second) + "\n")

log_file.close()

model.summary(print_fn=myprint)

log_file = open('results.log', 'a+')

log_file.write("Results:\t" + str(results) + "\n")

log_file.write("Images:\n")
log_file.write("\t" + pictures[0] + "\n")
log_file.write("\t" + pictures[1] + "\n")
log_file.write("\t" + pictures[2] + "\n")
log_file.write("\t" + pictures[3] + "\n")
log_file.write("\t" + pictures[4] + "\n")

log_file.close()

In [21]:
counts = Counter(types)
counts

Counter({'ISFJ': 1832,
         'INFJ': 1832,
         'INFP': 1832,
         'INTJ': 1832,
         'ISTP': 1832,
         'ESTP': 1832,
         'INTP': 1832,
         'ESFP': 1832,
         'ISTJ': 1832,
         'ENTP': 1832,
         'ENFP': 1832,
         'ENTJ': 1832,
         'ESTJ': 1832,
         'ISFP': 1832,
         'ENFJ': 1832,
         'ESFJ': 1832})